# Introduction to diffusion

## Diffusion equation 1-D with finite differences

Diffusion is a process that arises when a flow of a given quantity is induced by a gradient of its content, what is a direct response to second law of Thermodynamics. Under such circunstances, we often denote the flux vector $\vec{J}$ of quantity $u$ as given in the following equation, where $\nu$ is a transport coefficient. This empirical relationship has been shown to hold under several cases, constituting what is called *Fourier's law* in heat transfer and *Fick's first law* in mass transfer.

$$
    \vec{J}=-\nu{}\nabla{}u
$$

In this notebook we introduce the 1-D case of diffusion equation, thus the gradient is replaced by the derivative of $u$ over $x$ axis. To compute the time evolution of $u$ in a point of space in the absence of a source term, one simply aply the divergent operator (special case of Reynolds transport theorem without source) over the flux above which can be interpreted as

<center>
<bold>[ Inlet ] - [ Outlet ] + [ Creation == 0 ] = [ Accumulation rate ]</bold>
</center>

Applying this to the case where the creation rate is null (no source term) is equivalent to take the divergent of the flux. In the case the transport coefficient depends on position or in the transported quantity itself, this leads to the following nonlinear form of the diffusion equation, which we will study later in this notebook series:

$$
    \frac{\partial u}{\partial t}=\frac{\partial{}}{\partial{}x}\left[\nu(x, u)\frac{\partial{}u}{\partial{}x}\right]
$$

For the case of constant transport coefficient $\nu$, we finally get the expression that we study here:

$$
    \frac{\partial u}{\partial t}= \nu \frac{\partial^2{}u}{\partial{}x^2}
$$

### Second derivative computation

A second derivative of a function requires more information than the first one, since it represents the rate of change of a derivative. As a first strategy to compute the right-hand side of diffusion equation, let's take the Taylor series expansion of function $u$ around a given element $u_{i}$. We use this to produce both the next and previous elements in 1-D space, $u_{i+1}$ and $u_{i-1}$

$$
    u_{i+1} = u_i + \Delta x \frac{\partial u}{\partial x}\bigg|_i + \frac{\Delta x^2}{2} \frac{\partial ^2 u}{\partial x^2}\bigg|_i + \frac{\Delta x^3}{3!} \frac{\partial ^3 u}{\partial x^3}\bigg|_i + O(\Delta x^4)
$$

$$
    u_{i-1} = u_i - \Delta x \frac{\partial u}{\partial x}\bigg|_i + \frac{\Delta x^2}{2} \frac{\partial ^2 u}{\partial x^2}\bigg|_i - \frac{\Delta x^3}{3!} \frac{\partial ^3 u}{\partial x^3}\bigg|_i + O(\Delta x^4)
$$

As you may notice in the equations above, the sign of odd terms cancel. Adding up these expressions is a straighforward means of getting only even terms in the series, thus the second derivative.

$$
    u_{i+1} + u_{i-1} = 2u_i+\Delta x^2 \frac{\partial ^2 u}{\partial x^2}\bigg|_i + O(\Delta x^4)
$$

After manipulation for isolating the second derivative one produces the second order accurate scheme:

$$
    \frac{\partial ^2 u_{i}}{\partial x^2}=\frac{u_{i+1}-2u_{i}+u_{i-1}}{\Delta x^2} + O(\Delta x^2)
$$

### Explicit formulation

In the previous notebooks we have established the convention of using a superscript to denote the discrete time index. Computing the right-hand side of the diffusion equation at instant $n$ to predict $n+1$ produces the *explicit* formulation. For brevity we start using $\tau=\Delta{}t$ and $\delta=\Delta{}x$.

$$
    \frac{u_{i}^{n+1}-u_{i}^{n}}{\tau}=\nu\frac{u_{i+1}^{n}-2u_{i}^{n}+u_{i-1}^{n}}{\delta^2}
$$

Again, as we have done for the convection equation, $u^{n+1}$ can be promptly solved from $u^{n}$ at all positions:

$$
    u_{i}^{n+1}=u_{i}^{n}+\frac{\nu\tau}{\delta^2}(u_{i+1}^{n}-2u_{i}^{n}+u_{i-1}^{n})=
    (1-2\alpha)u_{i}^{n}+\alpha(u_{i+1}^{n}+u_{i-1}^{n})
    \qquad\text{where}\qquad{}\alpha=\frac{\nu{}\tau{}}{\delta^2}
$$

### Stability analysis

As we have seen in last notebook, the stability criterium for a formulation can be established through von Neumann analysis. To do so, we make use of the error scaling factors $\hat{\varepsilon}^{n}(k)\exp{(ikp\Delta{}x)}$ as a replacement of every $u$ in the above equation. Remember that for compatibility with imaginary unity, subindices were replaced by $p$ here, and the good value of $p$ must be used for each term. After manipulation we retrieve the amplification factor $g(k)$ that is independent of the position (try doing the full demonstration):

$$
g(k)=(1-2\alpha) + \alpha[\exp(ik\delta)+\exp(-ik\delta)]
$$

The right-hand side can be simplified through Euler's formula $\exp(\pm{}ix)=\cos(x)\pm{}i\sin(x)$. This leads to elimination of the sinuses from the equation, which now can be simplified as:

$$
g(k)=1+2\alpha\left[\cos(ik\delta)-1\right]
$$

Stability is assured when $\vert{}g(k)\vert\le{}1$ for any $k$. It can be shown (it is left for the reader to demonstrate), that this is possible for $\alpha\le\frac{1}{2}$. From now on we consider the derived criterium for the coding activities.

### Sample case

As we did when studying convection for the first time, our sample case will be started witn a hump in the middle of the domain, as per the statement below. The most important difference that is emphasized from now on is that since we are aware of stability criterium, time-step will be chosen to respect it. The code is provided with a flag for override the bounding of time-step allowing to investigate the effect of unstable regions of $\alpha$. In all cases, unity diffusion coefficient $\nu=1$ will be employed for simplicity.

$$
\begin{cases}
u=1 & x\in[0.75;\, 1.25]\\
u=0 & \text{elsewhere}
\end{cases}
$$

Using functions is a better practice than running code directly from cells, although you could loose some readability in notebooks. For the solution of real-world problems you will probably write modules and packages which contain the actual implementation. In the previous notebook we have already implemented a class for putting functionalities together and from now on the notebooks will focus in object-oriented or functional programming to avoid the risks of globals.

The next function provides the integration of the problem with the previously defined numerical scheme using NumPy array slicing to allow the computation of derivative in all internal domain elements. Notice that until the present we have not yet discussed boundary conditions and the next example is equivalent to a constant value over boundaries (Dirichlet condition, as we will see later in the series).

In [ ]:
from time import perf_counter
from IPython.core.display import HTML
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [ ]:
def problem_diffusion_linear(u, dx, t_end, alpha_max):
    """ Explicit solution of 1-D diffusion problem.
    
    Parameters
    ----------
    u : np.ndarray
        Array quantity over 1-D grid of step `dx`.
    dx : float
        Grid spacing in 1-D domain.
    t_end : float
        Final integration time.
    alpha_max : float
        Value of alpha for initial time-step calculation.
        
    Returns
    -------
    np.ndarray
        Solution array with quantity `u` at `t_end`.
    """
    # Constant diffusion coefficient.
    nu = 1.0
    
    # Compute dx**2 only once.
    dx2 = dx ** 2

    # Maximum stable time-step.
    dt = alpha_max * dx2 / nu
    
    # Initialize time.
    t = 0.0

    # Time loop until end.
    while t < t_end:
        # Bound time step for ending.
        dt = min(dt, t_end - t)
        
        # Increase solution time.
        t += dt 

        # Compute explicity solution
        alpha = nu * dt / dx2
        u[1:-1] += alpha * (u[2:] - 2.0 * u[1:-1] + u[:-2])
        
        if abs(t - t_end) <= 1.0e-08:
            return u

For allowing multiple calculations with same initial condition but different grids, we provide a simple function for allocation and computation of initial conditions and space discretization. This function will be used as a parameter of our simulation study, so different cases can be investigated.

In [ ]:
def get_initial_conditions_sample1(nx):
    """ Allocate x, u, and set initial state. """
    x, dx = np.linspace(0.0, 2.0, nx, retstep=True)
    u = np.zeros(nx, dtype=np.float64)
    u[((x >= 0.75) & (x <= 1.25))] = 1.0
    return x, u, dx

Finally we wrap the problem in a function to study the effect of alpha and integration time for a given fixed set of number of nodes in space. This function creates a plot of initial state and results for reference.

In [ ]:
def simulate_for_alpha(alpha, initialize, t_end=1.0e-03, bound_alpha=True):
    """ Simulate problem for a given `alpha`.
    
    Parameters
    ----------
    alpha : float
        Critical value of `alpha` to use in calculation.
    t_end : float
        Final integration time for studying effect of boundaries.
    bound_alpha : bool
        If `True`, ensure stability criterium.
    """
    if bound_alpha:
        alpha = min(alpha, 0.5)
    
    x, u, _ = initialize(5000)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=u, mode='lines', name='Initial',
                             line_shape='hvh'))
        
    t0 = perf_counter()
    for nx in [20, 200, 2000]:
        x, u, dx = initialize(nx)
        u = problem_diffusion_linear(u, dx, t_end, alpha)
        fig.add_trace(go.Scatter(x=x, y=u, mode='lines', name=F'nx={nx}',
                                 line_shape='hvh'))

    delay = perf_counter() - t0
    fig.update_layout(title=F'All simulations took {delay:.2f} s',
                      xaxis_title='Position [m]',
                      yaxis_title='Quantity [a.u.]')
    fig.show()

In the next cell we present the solution for the critical value of alpha that has been found in *von Neumann* stability analysis.

In [ ]:
simulate_for_alpha(0.5, get_initial_conditions_sample1)

As you observe in the plot above, with 2000 nodes in grid a pretty smooth solution starts to be produced with the given conditions. For 20 nodes, the problem is not even centered because of the placement of initial state in that case. The next cells are an invitation for you to experiment with different parameters in the simulation.

In [ ]:
# simulate_for_alpha(1.0, get_initial_conditions_sample1, bound_alpha=False)

In [ ]:
# simulate_for_alpha(0.5, get_initial_conditions_sample1, t_end=1.0e-01)

What if we have multiple humps (see them as particles being dissolved) in the system? Well, check below! This is a pretty common application case of diffusion in solid state and stationary liquids that is worth understanding.

In [ ]:
def get_initial_conditions_sample2(nx):
    """ Allocate x, u, and set initial state. """
    x, dx = np.linspace(0.0, 2.0, nx, retstep=True)
    u = np.zeros(nx, dtype=np.float64)
    u[((x >= 0.20) & (x <= 0.40))] = 2.0
    u[((x >= 0.70) & (x <= 1.00))] = 1.0
    u[((x >= 1.50) & (x <= 1.60))] = 3.0
    return x, u, dx

In [ ]:
simulate_for_alpha(0.5, get_initial_conditions_sample2, t_end=5.0e-03)

In [ ]:
HTML(open('notebook.css', 'r').read())